In [ ]:
# Install PyTorch Geometric and dependencies for PyTorch 2.0.1 + CUDA 11.8
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.1+cu118.html


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 962.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 124.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 118.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=12850be73662a174f93d52c7025ea170e8fd8b3ff3c645b5369abdbd8dba71cc
  Stored in directory: /root/.cache/pip/wheels/fc/5d/45/34fe9945d5e45e261134e72284395be36c2d4828af38e2b0fe
Successfully built lit
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

#Define project and data path
project_root = "/content/drive/MyDrive/Rebuilding_and_Modifying_GraphDTA"
data_path = f"{project_root}/data"

#Reload the proteins.csv
proteins_df = pd.read_csv(f"{data_path}/proteins.csv")

proteins_df.head()

,Protein_Index,Accession_Number,Gene_Name,Sequence
0,0,NP_055726.3,AAK1,MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQV...
1,1,NP_005148.2,ABL1(E255K)-phosphorylated,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...
2,3,NP_005148.2,ABL1(F317I)-phosphorylated,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...
3,5,NP_005148.2,ABL1(F317L)-phosphorylated,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...
4,7,NP_005148.2,ABL1(H396P)-phosphorylated,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...


In [ ]:
#Amino acid mapping
AMINO_ACIDS = 'ACDEFGHIKLMNPQRSTVWY'
AA_TO_IDX = {aa: idx + 1 for idx, aa in enumerate(AMINO_ACIDS)}


In [ ]:
#Building raw variable-length tensors

protein_tensors ={}

for _, row in proteins_df.iterrows():
  idx = row['Protein_Index']
  sequence = row['Sequence']
  indices = [AA_TO_IDX.get(aa, 0) for aa in sequence]
  protein_tensors[idx] = indices


In [ ]:
#Save as pickle file for future use

import pickle

#Save as a dictionary of raw index lists
with open(f"{data_path}/protein_index_sequences.pkl", "wb") as f:
  pickle.dump(protein_tensors, f)

print(f"Saved {len(protein_tensors)} raw protein sequences.")

Saved 433 raw protein sequences.


In [ ]:
import pandas as pd

#Define project and data path
project_root = "/content/drive/MyDrive/Rebuilding_and_Modifying_GraphDTA"
data_path = f"{project_root}/data"
affinity_df = pd.read_csv(f"{data_path}/drug_protein_affinity.csv")
print(affinity_df.head())

   Drug_Index  Protein_Index  Affinity
0           0              0  7.366532
1           0              1  5.000000
2           0              3  5.000000
3           0              5  5.000000
4           0              7  5.000000


In [ ]:
#Load graph tensors
import torch
import pickle

drug_graphs = torch.load(f"{data_path}/davis_drugs_graph.pt", weights_only=False)

#Load raw protein sequences
with open(f"{data_path}/protein_index_sequences.pkl", "rb") as f:
  protein_sequences = pickle.load(f)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

In [ ]:
from torch.utils.data import Dataset

class DTADataset(Dataset):
    def __init__(self, affinity_df, drug_graphs, protein_sequences):
        self.data = []
        for _, row in affinity_df.iterrows():
            d_idx = row["Drug_Index"]
            p_idx = row["Protein_Index"]
            y = row["Affinity"]

            if d_idx in drug_graphs and p_idx in protein_sequences:
                drug_graph = drug_graphs[d_idx]
                protein_seq = torch.tensor(protein_sequences[p_idx], dtype=torch.long)
                self.data.append((drug_graph, protein_seq, torch.tensor([y], dtype=torch.float)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


In [ ]:
#Build collate function

from torch.nn.utils.rnn import pad_sequence
from torch_geometric.data import Batch

def collate_fn(batch):
  drug_graphs, protein_seqs, labels = zip(*batch)
  drug_batch = Batch.from_data_list(drug_graphs)
  padded_proteins = pad_sequence(protein_seqs, batch_first=True, padding_value = 0)
  labels = torch.stack(labels)
  return drug_batch, padded_proteins, labels

In [ ]:
#Create train/val/test and loaders

from torch.utils.data import DataLoader, random_split

full_dataset = DTADataset(affinity_df, drug_graphs, protein_sequences)
train_size = int(0.8*len(full_dataset))
val_size = int(0.1*len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_set, val_set, test_set = random_split(full_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_set, batch_size = 512, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=512, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=512, collate_fn=collate_fn)


In [ ]:
#Output check

for drug_batch, protein_batch, affinity in train_loader:
  print("Drug graph batch:", drug_batch)
  print("Protein batch shape:", protein_batch.shape)
  print("Affinity batch:", affinity.shape)

Drug graph batch: DataBatch(x=[16279, 83], edge_index=[2, 35944], batch=[16279], ptr=[513])
Protein batch shape: torch.Size([512, 2527])
Affinity batch: torch.Size([512, 1])
Drug graph batch: DataBatch(x=[16510, 83], edge_index=[2, 36556], batch=[16510], ptr=[513])
Protein batch shape: torch.Size([512, 2549])
Affinity batch: torch.Size([512, 1])
Drug graph batch: DataBatch(x=[16262, 83], edge_index=[2, 36030], batch=[16262], ptr=[513])
Protein batch shape: torch.Size([512, 2549])
Affinity batch: torch.Size([512, 1])
Drug graph batch: DataBatch(x=[16391, 83], edge_index=[2, 36184], batch=[16391], ptr=[513])
Protein batch shape: torch.Size([512, 2549])
Affinity batch: torch.Size([512, 1])
Drug graph batch: DataBatch(x=[16513, 83], edge_index=[2, 36578], batch=[16513], ptr=[513])
Protein batch shape: torch.Size([512, 2527])
Affinity batch: torch.Size([512, 1])
Drug graph batch: DataBatch(x=[16400, 83], edge_index=[2, 36284], batch=[16400], ptr=[513])
Protein batch shape: torch.Size([512, 

In [ ]:
#GraphDTA model  (GCN + 1D CNN)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_max_pool

class GraphDTA(nn.Module):
  def __init__(self, num_atom_features=83, protein_vocab_size=21, protein_embed_dim=128, out_channel=128, kernel_size=8, drug_output_dim=128, protein_output_dim=128, fc_hidden_dims=[1024, 512], dropout_rate=0.2, output_dim=1):

    super(GraphDTA, self).__init__()

    #Drug encoder
    self.gcn1 = GCNConv(num_atom_features, num_atom_features*2)
    self.gcn2 = GCNConv(num_atom_features*2, num_atom_features*4)
    self.gcn3 = GCNConv(num_atom_features*4, num_atom_features*8)
    self.drug_fc1 = nn.Linear(num_atom_features*8, 1024)
    self.drug_fc2 = nn.Linear(1024, 512)
    self.drug_fc3 = nn.Linear(512, drug_output_dim)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(p=dropout_rate)

    #Protein encoder
    self.protein_embedding = nn.Embedding(protein_vocab_size, protein_embed_dim, padding_idx=0)
    self.conv1_protein = nn.Conv1d(protein_embed_dim, out_channels=out_channel, kernel_size=kernel_size)
    self.conv2_protein = nn.Conv1d(out_channel, protein_output_dim, kernel_size=kernel_size)
    self.protein_pool = nn.AdaptiveMaxPool1d(1) #output will be [batch_size, protein_output_dim, 1]

    #Fusion layers (MLP)
    self.fc1_combined = nn.Linear(drug_output_dim+protein_output_dim, fc_hidden_dims[0])
    self.dropout1 = nn.Dropout(p=dropout_rate)
    self.fc2_combined = nn.Linear(fc_hidden_dims[0], fc_hidden_dims[1])
    self.dropout2 = nn.Dropout(p=dropout_rate)
    self.out = nn.Linear(fc_hidden_dims[1], output_dim)

    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(dropout_rate) #General dropout rate

  def forward(self, drug_data, protein_seq):
    #Drug graph forward
    x_drug, edge_index_drug, batch_drug = drug_data.x, drug_data.edge_index, drug_data.batch
    x_drug = self.relu(self.gcn1(x_drug, edge_index_drug))
    x_drug = self.relu(self.gcn2(x_drug, edge_index_drug))
    x_drug = self.relu(self.gcn3(x_drug, edge_index_drug))
    drug_emb_pooled = global_max_pool(x_drug, batch_drug)

    drug_emb = self.relu(self.drug_fc1(drug_emb_pooled))
    drug_emb = self.dropout(drug_emb)
    drug_emb = self.relu(self.drug_fc2(drug_emb))
    drug_emb = self.dropout(drug_emb)
    drug_emb = self.relu(self.drug_fc3(drug_emb))
    drug_emb = self.dropout(drug_emb)

    #Protein sequence forward
    seq_emb = self.protein_embedding(protein_seq)
    seq_emb = seq_emb.permute(0, 2, 1)
    seq_conv = self.relu(self.conv1_protein(seq_emb))
    seq_conv = self.relu(self.conv2_protein(seq_conv))
    protein_emb = self.protein_pool(seq_conv).squeeze(-1)

    #Fusion
    combined_emb = torch.cat((drug_emb, protein_emb), dim = 1)
    x_combined = self.relu(self.fc1_combined(combined_emb))
    x_combined = self.dropout1(x_combined)
    x_combined = self.relu(self.fc2_combined(x_combined))
    x_combined = self.dropout2(x_combined)

    output = self.out(x_combined)
    if output.shape[-1] == 1:
      output = output.squeeze(-1)
    return output


In [ ]:
#Define training functions
def train(model, device, loader, optimizer, criterion):
  model.train()
  total_loss = 0
  for drug_graph, protein_seq, affinity in loader:
    drug_graph = drug_graph.to(device)
    protein_seq = protein_seq.to(device)
    affinity = affinity.to(device).squeeze()

    optimizer.zero_grad()
    output = model(drug_graph, protein_seq)
    loss = criterion(output, affinity)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    return total_loss / len(loader)


In [ ]:
#Define RMSE (PyTorch)
def rmse_torch(pred, true):
    return torch.sqrt(torch.mean((pred - true) ** 2)).item()

#Define CI(PyTorch)
def concordance_index_torch(y_true, y_pred):
    """Returns CI (pure PyTorch)"""
    concordant = 0.0
    permissible = 0.0
    n = len(y_true)

    for i in range(n):
        for j in range(i + 1, n):
            if y_true[i] != y_true[j]:
                permissible += 1
                if (y_pred[i] - y_pred[j]) * (y_true[i] - y_true[j]) > 0:
                    concordant += 1
    return concordant / permissible if permissible != 0 else 0.0


#Define evaluation function
def evaluate(model, device, val_loader, loader):
  model.eval()
  y_pred, y_true = [], []
  total_val_loss = 0
  num_samples = 0
  with torch.no_grad():
    for drug_graph, protein_seq, affinity in val_loader:
      drug_graph = drug_graph.to(device)
      protein_seq = protein_seq.to(device)
      affinity_label_device = affinity.to(device).squeeze()

      output = model(drug_graph, protein_seq)

      loss = criterion(output, affinity_label_device)
      total_val_loss += loss.item() * affinity_label_device.size(0)
      num_samples += affinity_label_device.size(0)

      y_pred.extend(output.detach().cpu().tolist())
      y_true.extend(affinity.squeeze().tolist())

  y_pred_tensor = torch.tensor(y_pred)
  y_true_tensor = torch.tensor(y_true)

  avg_val_loss = total_val_loss / num_samples

  metrics =  {
        'val_loss': avg_val_loss,
        'rmse': rmse_torch(y_pred_tensor, y_true_tensor),
        'ci': concordance_index_torch(y_true, y_pred)
    }
  return metrics

In [ ]:
#Initialize model, optimizer and loader

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = GraphDTA(num_atom_features=83, protein_vocab_size=21, protein_embed_dim=128, out_channel=128, kernel_size=8, drug_output_dim=128, protein_output_dim=128, fc_hidden_dims=[1024, 512], dropout_rate=0.2, output_dim=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
#Train the model

for epoch in range(1000):
  train_loss = train(model, device, train_loader, optimizer, criterion)
  val_metrics = evaluate(model, device, val_loader, criterion)
  print(f"Epoch {epoch:02d} | Loss: {train_loss: .4f} | Val RMSE: {val_metrics['rmse']:.4f} | CI: {val_metrics['ci']:.4f}")

Epoch 00 | Loss:  0.0152 | Val RMSE: 0.8810 | CI: 0.6361
Epoch 01 | Loss:  0.0181 | Val RMSE: 0.8796 | CI: 0.6367
Epoch 02 | Loss:  0.0162 | Val RMSE: 0.8792 | CI: 0.6373
Epoch 03 | Loss:  0.0162 | Val RMSE: 0.8807 | CI: 0.6380
Epoch 04 | Loss:  0.0126 | Val RMSE: 0.8872 | CI: 0.6387
Epoch 05 | Loss:  0.0154 | Val RMSE: 0.8953 | CI: 0.6393
Epoch 06 | Loss:  0.0205 | Val RMSE: 0.8914 | CI: 0.6399
Epoch 07 | Loss:  0.0191 | Val RMSE: 0.8821 | CI: 0.6404
Epoch 08 | Loss:  0.0167 | Val RMSE: 0.8777 | CI: 0.6410
Epoch 09 | Loss:  0.0159 | Val RMSE: 0.8793 | CI: 0.6415
Epoch 10 | Loss:  0.0152 | Val RMSE: 0.8787 | CI: 0.6421
Epoch 11 | Loss:  0.0201 | Val RMSE: 0.8774 | CI: 0.6426
Epoch 12 | Loss:  0.0170 | Val RMSE: 0.8784 | CI: 0.6431
Epoch 13 | Loss:  0.0179 | Val RMSE: 0.8821 | CI: 0.6436
Epoch 14 | Loss:  0.0167 | Val RMSE: 0.8855 | CI: 0.6439
Epoch 15 | Loss:  0.0151 | Val RMSE: 0.8888 | CI: 0.6444
Epoch 16 | Loss:  0.0139 | Val RMSE: 0.8877 | CI: 0.6446
Epoch 17 | Loss:  0.0163 | Val 

In [ ]:
#Save model weights

model_path = f"{data_path}/graphdta_baseline_weights.pt"
torch.save(model.state_dict(), model_path)
print(f"Saved model weights to {model_path}")

Saved model weights to /content/drive/MyDrive/Rebuilding_and_Modifying_GraphDTA/data/graphdta_baseline_weights.pt


In [ ]:
#Save full model

torch.save(model, f"{data_path}/graphdta_baseline_full_model.pt")
print(f"Saved full model.")

Saved full model.


In [ ]:
#Check the predictions of the model
import torch
from torch.utils.data import DataLoader
import random

sample_indices = random.sample(range(len(test_set)), 5)
samples = [test_set[i] for i in sample_indices]

In [ ]:
#Run predictions using the model

model.eval()
results = []

for drug_graph, protein_seq, true_affinity in samples:
  drug_graph = drug_graph.to(device)
  protein_seq = protein_seq.unsqueeze(0).to(device)

  drug_graph.batch = torch.zeros(drug_graph.num_nodes, dtype = torch.long).to(device)

  with torch.no_grad():
    pred_affinity = model(drug_graph, protein_seq).item()

  results.append({
      "True Affinity": round(true_affinity.item(), 4),
      "Predicted Affinity": round(pred_affinity, 4)
  })


In [ ]:
#Display Predictions

import pandas as pd

df_results = pd.DataFrame(results)

print(df_results)

   True Affinity  Predicted Affinity
0            5.0              4.9880
1            5.0              4.9972
2            5.0              4.8866
3            5.0              4.9412
4            5.0              4.8374
